#**RAG Fusion - The New Star of Search Technology**

In [1]:
# Installing the required dependencies
!pip install "langchain==0.0.344" "openai==0.28" pypdf tiktoken -q

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [3]:
# Importing Required Dependencies
import openai
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import LanceDB
import tiktoken

In [4]:
# Downloading the files
!wget https://ncert.nic.in/ncerts/l/leph202.pdf -O doc.pdf

--2024-09-12 10:15:37--  https://ncert.nic.in/ncerts/l/leph202.pdf
Resolving ncert.nic.in (ncert.nic.in)... 164.100.166.133
Connecting to ncert.nic.in (ncert.nic.in)|164.100.166.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3530023 (3.4M) [application/pdf]
Saving to: ‘doc.pdf’

doc.pdf             100%[===================>]   3.37M   453KB/s    in 8.9s    

2024-09-12 10:15:47 (389 KB/s) - ‘doc.pdf’ saved [3530023/3530023]



Splitting our documents into chunks.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

# Load the  pdf
pdf_folder_path = "/content/doc.pdf"

loader = PyPDFLoader(pdf_folder_path)
docs = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
documents = text_splitter.split_documents(docs)

In [6]:
embeddings = OpenAIEmbeddings()

###Using **LANCEDB** vector store for store and retreive embeddings.

In [7]:
from langchain.vectorstores import LanceDB
import lancedb

# lancedb as vectorstore
db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "documents",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)
vector_store = LanceDB.from_documents(documents, embeddings, connection=table)

Generating different queries relevant to the original query given by user.

In [14]:
def generate_queries_chatgpt(original_query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that generates multiple search queries based on a single input query.",
            },
            {
                "role": "user",
                "content": f"Generate multiple search queries related to: {original_query}",
            },
            {"role": "user", "content": "OUTPUT (4 queries):"},
        ],
    )
    generated_queries = response.choices[0].message.content.strip().split("\n")
    return generated_queries

Search relevant documents related to query in vector store.

In [15]:
def vector_search(query):
    search_results = {}
    retrieved_docs = vector_store.similarity_search(query)
    for i in retrieved_docs:
        search_results[i.page_content] = i.metadata["_distance"]
    search_results = {
        doc: score for doc, score in sorted(search_results.items(), key=lambda x: x[1])
    }
    return search_results

##Major Component of the RAG Fusion - **Reciprocal Rank Fusion Algorithm**
>This algorithm ranks documents on the basis of similarity to the query.

In [16]:
def reciprocal_rank_fusion(
    search_results_dict, k=60
):  # k=60 taken for optimum results according to paper.
    fused_scores = {}
    print("Initial individual search result ranks:")
    for query, doc_scores in search_results_dict.items():
        print(f"For query '{query}': {doc_scores}")

    for query, doc_scores in search_results_dict.items():
        for rank, (doc, score) in enumerate(
            sorted(doc_scores.items(), key=lambda x: x[1])
        ):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)
            print(
                f"Updating score for {doc} from {previous_score} to {fused_scores[doc]} based on rank {rank} in query '{query}'"
            )

    reranked_results = {
        doc: score
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    }
    print("Final reranked results:", reranked_results)
    return reranked_results

Generating output based on the reranked documents.

In [17]:
def generate_output(original_query, reranked_results):
    reranked_docs = [i for i in reranked_results.keys()]
    context = "\n".join(reranked_docs)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that answers user's questions based on the context provided.\nDo not make up an answer if you do not know it, stay within the bounds of the context provided, if you don't know the answer, say that you don't have enough information on the topic!",
            },
            {"role": "user", "content": f"CONTEXT: {context}\nQUERY: {original_query}"},
            {"role": "user", "content": "ANSWER:"},
        ],
    )

    response = response.choices[0].message.content.strip()
    return response

Now on to the final generation part with respect to queries given by user.

In [18]:
original_query = "Huygens Principle"
generated_queries = generate_queries_chatgpt(original_query)

In [19]:
generated_queries

['1. Huygens principle explanation',
 '2. Applications of Huygens principle',
 '3. Wave propagation according to Huygens principle',
 '4. Huygens-Fresnel principle comparison']

In [20]:
# Vector Search and document retreival for all the generated queries.
all_results = {}
for query in generated_queries:
    search_results = vector_search(query)
    all_results[query] = search_results

In [21]:
# Documents reranked accordig to RRF.
reranked_results = reciprocal_rank_fusion(all_results)

Initial individual search result ranks:
For query '1. Huygens principle explanation': {'Physics\n354small portion of the sphere can be considered as a plane and we have\nwhat is known as a plane wave  [Fig. 10.1(b)].\nNow, if we know the shape of the wavefront at t = 0, then Huygens\nprinciple allows us to determine the shape of the wavefront at a later\ntime τ. Thus, Huygens principle is essentially a geometrical construction,\nwhich given the shape of the wafefront at any time allows us to determine\nthe shape of the wavefront at a later time. Let us consider a diverging': 0.2861475646495819, 'In this chapter we will first discuss the original formulation of the\nHuygens principle  and derive the laws of reflection and refraction. In\nSections 10.4 and 10.5 , we will discuss the phenomenon of interference\nwhich is based on the principle of superposition. In Sec tion 10.6 we\nwill discuss the phenomenon of diffraction which is based on Huygens-\nFresnel principle. Finally in Sec tion

In [22]:
final_output = generate_output(original_query, reranked_results)

In [23]:
final_output

"Huygens' principle is a fundamental concept in wave optics that states that each point on a wavefront can be considered as a source of secondary spherical wavelets. These wavelets combine to form the new wavefront at a later time. This principle allows us to explain phenomena like reflection, refraction, interference, diffraction, and polarization of light waves."